## RECURRENT NEURAL NETWORKS USING LSTMS

In [95]:
from google.colab import drive,files
drive.mount('/content/drive')

Importing The Required Libraries

In [70]:
import pandas as pd;
import numpy as np;
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib


Loading The Preprocessed Dataset

In [72]:
dataset = pd.read_csv("/content/drive/MyDrive/RNN_Preprocessed_Dataset.csv")
dataset.head()

,Date,City,Value,City_Ampara,City_Anuradapura,City_Badulla,City_Battocolo,City_Colombo,City_Galle,City_Gampaha,...,City_Matale,City_Matara,City_Monaragala,City_Mulative,City_N eliya,City_Polonnaruwa,City_Puttalama,City_Rathnapura,City_Trincomalee,City_Vauniya
0,2010-01-01,Colombo,46,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-02-01,Colombo,59,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-03-01,Colombo,48,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-04-01,Colombo,70,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-05-01,Colombo,84,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Features Scaling

In [73]:
scaler = MinMaxScaler(feature_range=(0,1))
dataset["Value"]=scaler.fit_transform(dataset[['Value']])
dataset = dataset.sort_values('Date')

Preparing Data For Training

In [74]:
X_train = []
Y_train = []
sequence_length = 12
for i in range(len(dataset)-sequence_length):
    X_train.append(dataset.iloc[i:(i+sequence_length)].drop(columns=['Date','City','Value']).values)
    Y_train.append(dataset.iloc[i+sequence_length]['Value'])
X_train,Y_train = np.array(X_train), np.array(Y_train)

Split Into Training & Testing

In [75]:
length = int(len(dataset) *0.8)
X_train,X_test = X_train[:length],X_train[length:]
Y_train,Y_test = Y_train[:length],Y_train[length:]

Building The Neural Network

In [76]:
rnn = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],X_train.shape[2])),
    tf.keras.layers.LSTM(units=50,return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units=50,return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units=50,return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units=50),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1),
])

In [77]:
rnn.compile(optimizer="adam",loss="mean_squared_error")

Callback Function

In [78]:
callback = tf.keras.callbacks.EarlyStopping(patience=3,monitor="loss")

In [79]:
rnn.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_21 (LSTM)                       │ (None, 12, 50)              │          15,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 12, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_22 (LSTM)                       │ (None, 12, 50)              │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 12, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_23 (LSTM)                       │ (None, 12, 50)              │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 12, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_24 (LSTM)                       │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 76,051 (297.07 KB)

 Trainable params: 76,051 (297.07 KB)

 Non-trainable params: 0 (0.00 B)

In [80]:
rnn.fit(X_train,Y_train,epochs=100,batch_size=32,callbacks=[callback])

Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 13s 38ms/step - loss: 0.0023
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: 0.0017
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0019
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0016
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0024
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 0.0021
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.0017
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0018
Epoch 9/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0019
Epoch 10/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0015
Epoch 11/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0016
Epoch 12/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0020


In [ ]:

# Generate predictions
y_pred_train = rnn.predict(X_train)
y_pred_test = rnn.predict(X_test)


# Calculate additional metrics using sklearn
mse_test = mean_squared_error(Y_test, y_pred_test)
mse_train = mean_squared_error(Y_train, y_pred_train)

mae_train = mean_absolute_error(Y_train, y_pred_train)
mae_test = mean_absolute_error(Y_test, y_pred_test)

print(f"\nTrain MSE: {mse_train:.4f}")
print(f"\nTrain MSE: {mse_test:.4f}")
print(f"\nTrain MAE: {mae_train:.4f}")
print(f"\nTest MAE: {mae_test:.4f}")


86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step

Sklearn MSE: 0.0018
Sklearn MAE: 0.0210


Forecasting Predictions

In [98]:
def forecast_city(city_name,rnn,data,sequence,scaler):
    data = dataset[dataset['City'] == city_name]
    last_sequence = data.iloc[-sequence:].drop(columns=['Date','City','Value']).values

    forecasting = []
    for _ in range(12):
        forecasting_pred = rnn.predict(last_sequence.reshape(1,sequence,last_sequence.shape[1]))
        forecasting.append(forecasting_pred[0][0])
        forecasting_pred_expanded = np.hstack([
            forecasting_pred.reshape(-1),
            last_sequence[-1, 1:]
        ])
        last_sequence = np.vstack([last_sequence[1:], forecasting_pred_expanded])

    future_predictions = scaler.inverse_transform(np.array(forecasting).reshape(-1,1))
    return future_predictions

print(forecast_city('Ampara',rnn,dataset,sequence_length,scaler))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[[331.2024 ]
 [319.96704]
 [296.10782]
 [264.75122]
 [232.7117 ]
 [205.62114]
 [186.98666]
 [178.32448]
 [179.69435]
 [190.27032]
 [208.79343]
 [233.8631 ]]


Saving The Model

In [65]:
rnn.save("rnn.h5")

In [96]:
files.download("rnn.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>